## Setup

In [1]:
import numpy as np
import time
import json
import requests
import boto3
import os
import sagemaker
import io

In [2]:
from sagemaker.pytorch.model import PyTorchModel
from sagemaker.predictor import Predictor

In [3]:
from time import strftime,gmtime

In [4]:
import matplotlib.pyplot as plt
import cv2

In [5]:
import boto3
sns_client = boto3.client('sns')

/home/ec2-user/anaconda3/envs/pytorch_latest_p36/lib/python3.6/site-packages/boto3/compat.py:88: PythonDeprecationWarning: Boto3 will no longer support Python 3.6 starting May 30, 2022. To continue receiving service updates, bug fixes, and security updates please upgrade to Python 3.7 or later. More information can be found here: https://aws.amazon.com/blogs/developer/python-support-policy-updates-for-aws-sdks-and-tools/
  warnings.warn(warning, PythonDeprecationWarning)


In [6]:
from sagemaker import get_execution_role, Session, image_uris

role = get_execution_role()
sess = Session()
region = sess.boto_region_name
bucket = sess.default_bucket()
sm_client = boto3.client("sagemaker", region_name=region)

In [7]:
boto_session = boto3.session.Session()
sm_runtime = boto_session.client("sagemaker-runtime")

## Load model drive

In [8]:
#!pip install gdown

In [9]:
!gdown https://drive.google.com/uc?id=1HMgff-FV6qw1L0ywQZJ7ECa9VPq1bIoj

Downloading...
From: https://drive.google.com/uc?id=1HMgff-FV6qw1L0ywQZJ7ECa9VPq1bIoj
To: /home/ec2-user/SageMaker/Depth_Estimation/AdaBins/AdaBins_kitti.pt
100%|████████████████████████████████████████| 941M/941M [00:11<00:00, 84.3MB/s]


In [10]:
!mv AdaBins_kitti.pt model.pth

### Create model archive

In [11]:
model_archive_name = 'Adabins-async.tar.gz'

In [12]:
!tar -cvzf {model_archive_name} model.pth code/

model.pth
code/
code/models/
code/models/unet_adaptive_bins.py
code/models/layers.py
code/models/miniViT.py
code/models/__init__.py
code/model_io.py
code/requirements.txt
code/inference.py
code/.ipynb_checkpoints/


In [13]:
# model package tarball (model artifact + inference code)
model_url = sess.upload_data(path=model_archive_name, key_prefix='AdaBins_model')
print('model uploaded to: {}'.format(model_url))

model uploaded to: s3://sagemaker-ca-central-1-333752261573/AdaBins_model/Adabins-async.tar.gz


### Create model and test inference

In [14]:
framework_version = '1.7.1'
py_version = 'py36'
env= {
            'TS_MAX_REQUEST_SIZE': '100000000', #default max request size is 6 Mb for torchserve, need to update it to support the 70 mb input payload
            'TS_MAX_RESPONSE_SIZE': '100000000',
            'TS_DEFAULT_RESPONSE_TIMEOUT': '1000'
        }

sm_model = PyTorchModel(model_data=model_url,
                               framework_version=framework_version,
                               role=role,
                               sagemaker_session=sess,
                               entry_point='inference.py',
                               source_dir= 'code',
                               env=env,
                               py_version=py_version
                              )

In [15]:
instance_type = 'ml.g4dn.xlarge'
uncompiled_predictor = sm_model.deploy(initial_instance_count=1, instance_type=instance_type)

----------!

# Prediction

In [16]:
import boto3
import requests
client = boto3.client('sagemaker-runtime', region_name=region)
content_type = 'application/x-image'

In [107]:
# test sample 2
path= 'test_data/frame54.jpg'
img = cv2.imread(path)
img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

body = io.open(path, 'rb')

In [108]:
t0 = time.time()
rv = client.invoke_endpoint(EndpointName=uncompiled_predictor.endpoint_name, Body=body, ContentType=content_type)
t1 = time.time()

time_elapsed = (t1-t0)*1000
print(time_elapsed)

602.1256446838379


In [109]:
predictions= json.loads(rv['Body'].read().decode())

In [110]:
type(predictions)

list

In [111]:
predictions_arr = np.array(predictions)

In [112]:
predictions_arr.shape

(1, 1, 480, 640)

# Measure invocation time 

In [33]:
inferenceTime_list=[]
for i in range(10):
    body = io.open(path, 'rb')
    t0 = time.time()
    rv = client.invoke_endpoint(EndpointName=uncompiled_predictor.endpoint_name, Body=body, ContentType=content_type)
    t1 = time.time()

    time_elapsed = (t1-t0)*1000
    inferenceTime_list.append(time_elapsed)
    print(time_elapsed)

627.171516418457
653.822660446167
683.9485168457031
674.1800308227539
655.482292175293
665.8425331115723
646.6827392578125
651.9558429718018
659.8081588745117
652.277946472168


In [34]:
np.mean(inferenceTime_list)

657.117223739624

# Async endpoint

In [14]:
from sagemaker.async_inference.async_inference_config import AsyncInferenceConfig

bucket= 'gulogulo-inference-results'

async_config = AsyncInferenceConfig(
                output_path= f"s3://{bucket}/output",
                max_concurrent_invocations_per_instance=2,)


In [15]:
framework_version = '1.7.1'
py_version = 'py36'
env= {
            'TS_MAX_REQUEST_SIZE' : '1000000000', #default max request size is 6 Mb for torchserve, need to update it to support the 70 mb input payload
            'TS_MAX_RESPONSE_SIZE': '1000000000',
            'TS_DEFAULT_RESPONSE_TIMEOUT': '1000'
        }

sm_model = PyTorchModel(model_data=model_url,
                               framework_version=framework_version,
                               role=role,
                               sagemaker_session=sess,
                               entry_point='inference.py',
                               source_dir= 'code-async',
                               env=env,
                               py_version=py_version
                              )

In [16]:
instance_type = 'ml.g4dn.xlarge'

async_uncompiled_predictor = sm_model.deploy(async_inference_config=async_config,
                                       initial_instance_count=1,
                                       instance_type=instance_type)

----------!

# Test endpoint

In [19]:
endpoint_name= ''

In [26]:
input_1_s3_location= ''

### Invoke async endpoint

In [ ]:
response = sm_runtime.invoke_endpoint_async(
    EndpointName=endpoint_name, 
    InputLocation=input_1_s3_location)
output_location = response['OutputLocation']
print(f"OutputLocation: {output_location}")